In [1]:
%load_ext autoreload
%autoreload 2

from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, OKXRestAPI, GateIORestAPI
from jaref_bot.config.credentials import host, user, password, db_name

import pandas as pd
import numpy as np
import polars as pl

from datetime import datetime, timezone, UTC
import requests
from dataclasses import dataclass
import hmac
import hashlib
import base64
import json
import random
import string
from time import sleep
from decimal import Decimal
import pickle

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

#### exchanges api

In [ ]:
def get_missing_instr_info(exchange, market_type, symbol):
    exc_dict = {'bybit': BybitRestAPI, 'okx': OKXRestAPI, 'gate': GateIORestAPI}
    client = exc_dict[exchange](market_type)
    token = client._create_symbol_name(symbol)
    
    return client.get_instrument_data(symbol=token)

In [ ]:
def get_order_status(exchange, market_type, token):
    order_in_pending = db_manager.order_exists(table_name='pending_orders', exchange=exchange, market_type=market_type, token=token)
    order_in_current = db_manager.order_exists(table_name='current_orders', exchange=exchange, market_type=market_type, token=token)

    if not (order_in_pending or order_in_current):
        status = None
    elif order_in_pending and order_in_current:
        status = 'adding'
    elif order_in_pending and not order_in_current:
        status = 'placed'
    elif not order_in_pending and order_in_current:
        status = 'live'
    return status

In [ ]:
market_fees = {'bybit_spot': 0.001, 'bybit_linear': 0.00055, 'okx_spot': 0.001, 'okx_linear': 0.0005,
               'gate_spot': 0.002, 'gate_linear': 0.0005}

In [ ]:
# ====================================
# Инициация нужных криптобирж, рынков и БД
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
exc_manager.add_market("bybit_spot", BybitRestAPI('spot'))
exc_manager.add_market("okx_linear", OKXRestAPI('linear'))
exc_manager.add_market("okx_spot", OKXRestAPI('spot'))
exc_manager.add_market("gate_linear", GateIORestAPI('linear'))
exc_manager.add_market("gate_spot", GateIORestAPI('spot'))

db_params = {'host': host, 'user': user, 'password': password, 'database': db_name}
db_manager = DBManager(db_params)
print('auto copy from current_data to market_data', db_manager.get_auto_copy_trigger_state())

In [ ]:
coin_information = exc_manager.get_instrument_data()

In [ ]:
class OkxAccount(OkxClient):
    def __init__(self, demo=True, debug=True):
        OkxClient.__init__(self, demo=demo, debug=debug)

    # Get Balance
    def get_account_balance(self, symbol=''):
        params = {}
        if symbol:
            params['ccy'] = symbol
        return self._request_with_params(method='GET', request_path='/api/v5/account/balance', params=params)

    # Get Positions
    def get_positions(self, market_type='', symbol=''):
        if market_type == 'linear':
            market_type = 'SWAP'
        params = {'instType': market_type, 'instId': symbol}
        return self._request_with_params(method='GET', request_path='/api/v5/account/positions', params=params)

    def get_leverage(self, market_type, symbol, margin_mode):
        sym = self._create_symbol_name(market_type, symbol)
        params = {'instId': sym, 'mgnMode': margin_mode}
        return self._request_with_params('GET', request_path='/api/v5/account/leverage-info', params=params)
    
    def set_leverage(self, market_type, symbol, leverage, margin_mode, ccy='', pos_side=''):
        sym = self._create_symbol_name(market_type, symbol)
        params = {'lever': leverage, 'mgnMode': margin_mode, 'instId': sym, 'ccy': ccy, 'posSide': pos_side}
        return self._request_with_params('POST', request_path='/api/v5/account/set-leverage', params=params)

    def get_fee_rates(self, symbol, instId='', uly='', category='', instFamily=''):
        sym = self._create_symbol_name(market_type, symbol)
        params = {'instType': instType, 'instId': sym, 'uly': uly, 'category': category, 'instFamily': instFamily}
        return self._request_with_params(GET, FEE_RATES, params)

In [ ]:
account_client = OkxAccount(demo=True, debug=False)

In [ ]:
account_client.get_leverage(market_type='linear', symbol='1INCH', margin_mode='isolated')

In [ ]:
account_client.set_leverage(market_type='linear', symbol='BTC', leverage=1, margin_mode='isolated')

In [ ]:
# Закрытие позиции на спотовом рынке. Надо иметь в виду, что купленное кол-во токенов будет на размер fee меньше 
#   запрошенного. То есть при закрытии надо из qty вычитать fee, и это значение отправлять в заявку

# trade_client.market_order(market_type='spot', symbol='ADA', side='sell', order_type='market', qty=19.98)

#### current profit

In [ ]:
import pandas as pd

try:
    current_result_df = pd.read_parquet('./data/current_result_df.parquet')
    for token in current_result_df['token'].unique():
        if token == 'GMT_USDT':
            continue
        temp_df = current_result_df[current_result_df['token'] == token].copy()
    
        temp_df[['profit', 'diff']] = temp_df[['profit', 'diff']].astype(float)
        temp_df[['profit', 'diff']].plot(figsize=(12, 3), title=token);
except FileNotFoundError:
    print('Нет текущих ордеров')

In [ ]:
current_result_df

#### Поиск наилучших условий для входа и выхода

In [ ]:
%load_ext autoreload
%autoreload 2

from jaref_bot.db.postgres_manager import DBManager
import pandas as pd
import numpy as np
from jaref_bot.config.credentials import host, user, password, db_name
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from decimal import Decimal
from tqdm.notebook import tqdm
from datetime import datetime

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

In [ ]:
market_fees = {'bybit_spot': 0.0018, 'bybit_linear': 0.001, 'okx_spot': 0.001, 'okx_linear': 0.0005,
               'gate_spot': 0.002, 'gate_linear': 0.0005}

In [ ]:
db_manager.clear_old_data(table='market_data_5s', column='bucket', expiration_time=48, units='hours')

In [ ]:
# Список нужных мне криптобирж
exchanges = ('bybit', 'okx', 'gate')

# Создаём пустой датафрейм для хранения результатов
stats_df = pd.DataFrame(columns = ['token', 'long_exc', 'short_exc', 'mean', 'std', 'cmean', 'cstd'])

print('Загружаем список уникальных токенов...')
tokens = db_manager.get_unique_tokens()
print('Обновляем статистические данные...')
for token in tqdm(tokens):
    tdf = db_manager.get_token_history(token)
    tdf = tdf.sort_values(by='bucket').reset_index(drop=True)
    tdf[['avg_bid', 'avg_ask']] = tdf[['avg_bid', 'avg_ask']].astype(float)

    for long_exc in exchanges:
        for short_exc in exchanges:
            lm_ask_price = pd.DataFrame()
            sm_bid_price = pd.DataFrame()
            
            if long_exc == short_exc:
                continue
            
            long_mask = (tdf['exchange'] == long_exc) & (tdf['market_type'] == 'linear')
            short_mask = (tdf['exchange'] == short_exc) & (tdf['market_type'] == 'linear')
            
            lm_ask_price = tdf[long_mask][['bucket', 'exchange', 'market_type', 'avg_ask']]
            sm_bid_price = tdf[short_mask][['bucket', 'exchange', 'market_type', 'avg_bid']]
            temp_df = lm_ask_price.merge(sm_bid_price, on='bucket', suffixes=('_long', '_short'))
            temp_df['diff'] = (temp_df['avg_bid'] / temp_df['avg_ask'] - 1) * 100
    
            minv, maxv, meanv, stdv = temp_df['diff'].agg(['min', 'max', 'mean', 'std'])

            lm_bid_price = tdf[long_mask][['bucket', 'exchange', 'market_type', 'avg_bid']]
            sm_ask_price = tdf[short_mask][['bucket', 'exchange', 'market_type', 'avg_ask']]
            close_df = lm_bid_price.merge(sm_ask_price, on='bucket', suffixes=('_long', '_short'))
            close_df['diff'] = (close_df['avg_bid'] / close_df['avg_ask'] - 1) * 100

            cminv, cmaxv, cmean, cstd = close_df['diff'].agg(['min', 'max', 'mean', 'std'])

    
            stats_df.loc[len(stats_df)] = {'token': token, 'long_exc': long_exc, 'short_exc': short_exc, 
                                           'mean': meanv, 'std': stdv, 'cmean': cmean, 'cstd': cstd}

stats_df.dropna(inplace=True)
stats_df.reset_index(drop=True, inplace=True)
db_manager.clear_table('stats_data')
db_manager.update_stats(stats_df)
ct = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('last updated:', ct)

#### Графики

In [ ]:
def show_diff_in(token, long_exc, short_exc, open_coef, close_coef, start_time=None):
    try:
        tdf = db_manager.get_token_history(token)
        print(f'Первая запись: {tdf.iloc[0]['bucket']}')
        print(f'Последняя запись: {tdf.iloc[-1]['bucket']}')
        
        tdf[['avg_bid', 'avg_ask']] = tdf[['avg_bid', 'avg_ask']].astype(float)
        if start_time:
            tdf = tdf[tdf['bucket'] > start_time]
        
        long_mask = (tdf['exchange'] == long_exc) & (tdf['market_type'] == 'linear')
        short_mask = (tdf['exchange'] == short_exc) & (tdf['market_type'] == 'linear')
        
        lm_ask_price = tdf[long_mask][['bucket', 'exchange', 'market_type', 'avg_ask']].sort_values('bucket').reset_index(drop=True)
        sm_bid_price = tdf[short_mask][['bucket', 'exchange', 'market_type', 'avg_bid']].sort_values('bucket').reset_index(drop=True)
        open_df = lm_ask_price.merge(sm_bid_price, on='bucket', suffixes=('_long', '_short'))
        open_df['diff'] = (open_df['avg_bid'] / open_df['avg_ask'] - 1) * 100
        open_diff = open_df['diff'].iloc[-1]
    except IndexError:
        print('Такой монеты в истории данных не существует.')
        return None

    lm_bid_price = tdf[long_mask][['bucket', 'exchange', 'market_type', 'avg_bid']].sort_values('bucket').reset_index(drop=True)
    sm_ask_price = tdf[short_mask][['bucket', 'exchange', 'market_type', 'avg_ask']].sort_values('bucket').reset_index(drop=True)
    close_df = lm_bid_price.merge(sm_ask_price, on='bucket', suffixes=('_long', '_short'))
    close_df['diff'] = (close_df['avg_bid'] / close_df['avg_ask'] - 1) * 100
    close_diff = close_df['diff'].iloc[-1]
    
    ominv, omaxv, omeanv, ostdv = open_df['diff'].agg(['min', 'max', 'mean', 'std'])
    omax_diff = omaxv - ominv
    odev = (open_diff - omeanv) / ostdv
    o_edge = omeanv + open_coef * ostdv

    cminv, cmaxv, cmeanv, cstdv = close_df['diff'].agg(['min', 'max', 'mean', 'std'])
    cmax_diff = cmaxv - cminv
    cdev = (close_diff - cmeanv) / cstdv
    c_edge = cmeanv + close_coef * cstdv
    
    width = 120
    print(f"{('=' * 60 + ' ' + token + ' ' + '=' * 60).center(width)}")
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows=2, figsize=(16, 6),
                               gridspec_kw={'width_ratios': [3, 1]})
    
    ax1.plot(open_df['bucket'], open_df['diff'])
    ax1.set_title(f'{token}. {long_exc=}; {short_exc=}.   mean: {omeanv:.3f}, std: {ostdv:.3f}; diff: {open_diff:.3f}, {o_edge=:.3f}')
    ax1.axhline(y=omeanv, color='black', linestyle='-') # mean
    # ax1.axhline(y=0.4, color='r', linestyle='-') # low bound
    ax1.axhline(y=omeanv + open_coef * ostdv, color='g', linestyle='-'); # high bound

    sns.kdeplot(data=open_df['diff'], ax=ax2, fill=True, color='blue')
    # ax2.set_title("Гистограмма")
    
    ax3.plot(close_df['bucket'], close_df['diff'])
    ax3.set_title(f'mean: {cmeanv:.3f}, std: {cstdv:.3f}; diff: {close_diff:.3f}, dev: {cdev:.3f}; {c_edge=:.3f}')
    ax3.axhline(y=cmeanv, color='black', linestyle='-') # mean
    ax3.axhline(y=0, color='r', linestyle='-') # low bound
    ax3.axhline(y=cmeanv + close_coef * cstdv, color='g', linestyle='-'); # high bound

    sns.kdeplot(data=close_df['diff'], ax=ax4, fill=True, color='blue')
    
    plt.tight_layout()
    plt.show()
    print('=' * (width + len(token) + 2))

In [ ]:
def show_diff_out(token, upper_coef):
    # Определяем рынки, на которых открыт ордер
    current_orders = db_manager.get_table('current_orders')
    orders = current_orders[current_orders['token'] == token]
    
    long_market = orders[orders['order_side'] == 'buy']
    short_market = orders[orders['order_side'] == 'sell']
    
    long_exc = long_market['exchange'].item()
    short_exc = short_market['exchange'].item()
    long_mt = long_market['market_type'].item()
    short_mt = short_market['market_type'].item()
    
    # Определяем цены на момент открытия и начальный diff
    long_open_price = long_market['price'].item().normalize()
    short_open_price =  short_market['price'].item().normalize()
    open_diff = round((short_open_price / long_open_price - 1) * 100, 3)

    tdf = db_manager.get_token_history(token)
    tdf[['avg_bid', 'avg_ask']] = tdf[['avg_bid', 'avg_ask']].astype(float)
    
    long_mask = (tdf['exchange'] == long_exc) & (tdf['market_type'] == long_mt)
    short_mask = (tdf['exchange'] == short_exc) & (tdf['market_type'] == short_mt)
    tdf = tdf[long_mask | short_mask]
    
    long_mask = long_mask.reindex(tdf.index, fill_value=False)
    short_mask = short_mask.reindex(tdf.index, fill_value=False)
    
    lm_sell_price = tdf[long_mask][['bucket', 'avg_bid']]
    sm_buy_price = tdf[short_mask][['bucket', 'avg_ask']]
    close_df = lm_sell_price.merge(sm_buy_price)
    close_df['diff'] = (close_df['avg_bid'] / close_df['avg_ask'] - 1) * 100

    minv, maxv, meanv, stdv = close_df['diff'].astype(float).agg(['min', 'max', 'mean', 'std'])
    
    close_diff = close_df['diff'].iloc[-1]
    deviation = (close_diff - meanv) / stdv

    # Профит
    long_market_fee = long_market['usdt_fee'].iloc[0].normalize()
    short_market_fee = short_market['usdt_fee'].iloc[0].normalize()
    qty = short_market['qty'].iloc[0].normalize()
    
    long_fee = Decimal(market_fees[long_exc + '_linear'])
    short_fee = Decimal(market_fees[short_exc + '_linear'])
    
    def get_curr_profit(row):
        ct = row['bucket']
        long_market_profit = Decimal(row['avg_bid'])  - long_open_price 
        short_market_profit = short_open_price  - Decimal(row['avg_ask'])
        profit = qty * (long_market_profit + short_market_profit)    
        # print(f'{ct}: lm_profit: {qty * long_market_profit:.3f}, sm_profit: {qty * short_market_profit:.3f}; {row['avg_bid']=}, {row['avg_ask']=}')
        return round(profit, 4)
    
    close_df['profit'] = close_df.apply(get_curr_profit, axis=1)
    curr_profit = close_df['profit'].iloc[-1]
    order_time = orders.iloc[0]['created_at']
    new_df = close_df[close_df['bucket'] > order_time.tz_localize(None)].reset_index(drop=True)
        
    plt.figure(figsize=(18, 2))
    plt.title(f'{token}. Diff out. {meanv=:.3f}, {stdv=:.3f}')
    plt.plot(close_df['bucket'], close_df['diff']);
    plt.axhline(y=meanv, color='black', linestyle='-') # mean
    plt.axhline(y=meanv + upper_coef * stdv, color='g', linestyle='-'); # high bound
    plt.axhline(y=0, color='r', linestyle='-')
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 3), gridspec_kw={'width_ratios': [1, 1]})
    ax2.set_title(f'Profit. {curr_profit=:.3f}')
    ax2.plot(new_df['bucket'], new_df['profit']);
    ax2.axhline(y=0, color='black', linestyle='-')
    ax2.axhline(y=new_df['profit'].max(), color='g', linestyle='-')

    ax1.set_title(f'current diff: {close_diff:.3f}, {deviation=:.1f}')
    ax1.plot(new_df['bucket'], new_df['diff'])
    ax1.axhline(y=meanv, color='black', linestyle='-') # mean
    ax1.axhline(y=meanv + upper_coef * stdv, color='g', linestyle='-'); # high bound
    ax1.axhline(y=0, color='r', linestyle='-')

    plt.tight_layout()
    plt.show()

In [ ]:
token = 'WCT_USDT'
long_exc = 'bybit'
short_exc = 'gate'

In [ ]:
show_diff_in(token=token, long_exc=long_exc, short_exc=short_exc, open_coef=1.5, close_coef=1.5, start_time='2025-04-10 11:20')

In [ ]:
from jaref_bot.strategies.arbitrage import get_best_prices
from jaref_bot.db.redis_manager import RedisManager
import polars as pl
redis_manager = RedisManager('orderbooks')

df = redis_manager.get_orderbooks(1).filter(pl.col('symbol') == token)
long_open = df.filter(pl.col('exchange') == long_exc).select('askprice_0').item()
short_open = df.filter(pl.col('exchange') == short_exc).select('bidprice_0').item()
open_diff = (short_open / long_open - 1) * 100

print(long_open, short_open, open_diff)

In [ ]:
# Проверка времени последней записи
redis_manager.get_orderbooks(1).filter(pl.col('symbol') == token)

In [ ]:
current_orders = db_manager.get_table('current_orders')
for i, row in current_orders.pivot(index='token', columns='order_side', values='exchange').iterrows():
    token = i
    long_exc = row['buy']
    short_exc = row['sell']
    print('=' * 140)
    show_diff_out(token=token, upper_coef=2.0)
    print('=' * 140)
    print()

In [ ]:
long_close = 0.02656
short_close = 0.0262
close_diff = (long_close / short_close - 1) * 100

long_profit = (long_close / long_open - 1) * 100 - 0.0015
short_profit = (short_open / short_close - 1) * 100 - 0.0015
fund_profit = 0.005	+ 0.242987
profit = long_profit + short_profit + fund_profit

In [ ]:
close_diff, profit

In [1]:
# import polars as pl
# token = 'SERAPH_USDT'
# tdf = db_manager.get_token_history(token)
# tdf = pl.from_pandas(tdf)
# tdf = tdf.with_columns([pl.col('avg_bid').cast(float), 
#                   pl.col('avg_ask').cast(float)])
# long_exc = 'bybit'
# short_exc = 'gate'

# long_market = tdf.filter(pl.col('exchange') == long_exc).drop('market_type')
# short_market = tdf.filter(pl.col('exchange') == short_exc).drop('market_type')

# main_df = long_market.join(short_market, on=('bucket'), suffix='_short'
#     ).drop('token_short'
#     ).rename({'exchange': 'long_exc', 'exchange_short': 'short_exc', 'avg_bid': 'lm_bid', 'avg_ask': 'lm_ask',
#              'avg_bid_short': 'sm_bid', 'avg_ask_short': 'sm_ask'}
#     ).with_columns([
#         ((pl.col('sm_bid') / pl.col('lm_ask') - 1) * 100).alias('open_diff'),
#         ((pl.col('lm_bid') / pl.col('sm_ask') - 1) * 100).alias('close_diff')
#     ]).with_columns([
#         pl.col("open_diff").mean().alias("o_mean"),
#         pl.col("open_diff").std().alias("o_std"),
#         pl.col("close_diff").mean().alias("c_mean"),
#         pl.col("close_diff").std().alias("c_std"),
#     ])
# dev_in = 2.5
# dev_out = 1.0

# main_df.with_columns([
#     pl.col('o_mean') + dev_in * pl.col('o_std') 
# ])